In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
df_atraso = pd.read_csv(r'BD Tratado\atraso.csv',sep = ';')
df_atraso = df_atraso[['MATRÍCULA', 'DATA', 'ATRASO']]
df_atraso = df_atraso.sort_values(['MATRÍCULA', 'DATA'])
df_atraso['DATA'] = pd.to_datetime(df_atraso['DATA'])
df_atraso = df_atraso[df_atraso['ATRASO'] != 0]
df_atraso.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97402 entries, 5422 to 761014
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   MATRÍCULA  97402 non-null  int64         
 1   DATA       97402 non-null  datetime64[ns]
 2   ATRASO     97402 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 3.0 MB


In [3]:
df_atraso.head()

,MATRÍCULA,DATA,ATRASO
5422,3108,2020-01-03,1.0
13922,3108,2020-01-10,72.0
67987,3108,2020-01-20,46.0
86047,3108,2020-01-21,47.0
27640,3108,2020-01-22,32.0


In [4]:
df_carga = pd.read_csv(r'BD Tratado\carga.csv',sep=';')
print(df_carga.head())

   MATRÍCULA        DATA  Carga    DATA FIM
0       3108  2020-01-02    9.0  2021-01-31
1       3143  2020-01-08    9.0  2021-01-31
2      10107  2020-01-02    9.0  2021-01-31
3      10137  2020-01-02    9.0  2021-01-31
4      10182  2020-01-02    9.0  2020-11-12


In [5]:
df_carga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8020 entries, 0 to 8019
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MATRÍCULA  8020 non-null   int64  
 1   DATA       8020 non-null   object 
 2   Carga      7991 non-null   float64
 3   DATA FIM   8020 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 250.8+ KB


In [6]:
df_jornada = pd.read_csv(r'BD Tratado\horarios.csv', sep = ';')
df_jornada.drop(['Unnamed: 0', 'HORÁRIO'], axis=1, inplace=True)
df_jornada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   MATRÍCULA        164 non-null    int64
 1   Jornada - 5x2    164 non-null    int64
 2   Jornada - 6x1    164 non-null    int64
 3   Jornada - 12x36  164 non-null    int64
 4   Jornada - movel  164 non-null    int64
 5   Jornada - fixo   164 non-null    int64
dtypes: int64(6)
memory usage: 7.8 KB


In [7]:
df_jornada.head()

,MATRÍCULA,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo
0,205827,0,1,0,1,0
1,90293,1,0,0,0,1
2,200400,1,0,0,0,1
3,200570,1,0,0,0,1
4,200981,0,1,0,1,0


In [8]:
df_noturno = pd.read_csv(r'BD Tratado\noturno.csv',sep=';')
df_noturno = df_noturno.sort_values(['MATRÍCULA', 'DATA'])
df_noturno['DATA FIM'] = np.where(df_noturno['MATRÍCULA'].shift(-1) == df_noturno['MATRÍCULA'], df_noturno['DATA'].shift(-1), '2021-01-31')
df_noturno['DATA FIM'] = pd.to_datetime(df_noturno['DATA FIM'])
df_noturno['DATA'] = pd.to_datetime(df_noturno['DATA'])
df_noturno.head(10)

,MATRÍCULA,DATA,NOTURNO,DATA FIM
11163,3108,2020-02-13,0,2020-02-14
11127,3108,2020-02-14,0,2020-06-10
9616,3108,2020-06-10,0,2020-06-30
9265,3108,2020-06-30,0,2020-07-10
9097,3108,2020-07-10,0,2020-07-30
8760,3108,2020-07-30,0,2020-08-26
8263,3108,2020-08-26,0,2021-01-29
3167,3108,2021-01-29,0,2021-01-31
11398,3143,2020-01-24,0,2021-01-29
2775,3143,2021-01-29,0,2021-01-31


In [9]:
df = df_atraso.merge(df_jornada, how = 'left', on = 'MATRÍCULA')
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo
0,3108,2020-01-03,1.0,NaN,NaN,NaN,NaN,NaN
1,3108,2020-01-10,72.0,NaN,NaN,NaN,NaN,NaN
2,3108,2020-01-20,46.0,NaN,NaN,NaN,NaN,NaN
3,3108,2020-01-21,47.0,NaN,NaN,NaN,NaN,NaN
4,3108,2020-01-22,32.0,NaN,NaN,NaN,NaN,NaN


In [10]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_carga.to_sql('df_carga', conn, index=False)

qry = '''
    select  distinct
        t1.*,
        t2.Carga
        
    from
        df as t1
        left join df_carga as t2
            on t1.MATRÍCULA = t2.MATRÍCULA 
            and t1.DATA >= t2.DATA 
            and t1.DATA <= t2.[DATA FIM]
        '''
df = pd.read_sql_query(qry, conn)
df.head()

C:\Users\gabri\anaconda3\envs\faculdade\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,Carga
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,9.0
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,9.0
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,9.0
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,9.0
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,9.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97402 entries, 0 to 97401
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MATRÍCULA        97402 non-null  int64  
 1   DATA             97402 non-null  object 
 2   ATRASO           97402 non-null  float64
 3   Jornada - 5x2    2703 non-null   float64
 4   Jornada - 6x1    2703 non-null   float64
 5   Jornada - 12x36  2703 non-null   float64
 6   Jornada - movel  2703 non-null   float64
 7   Jornada - fixo   2703 non-null   float64
 8   Carga            97273 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 6.7+ MB


In [12]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_noturno.to_sql('df_noturno', conn, index=False)

qry = '''
    select  distinct
        t1.*,
        t2.[NOTURNO]
        
    from
        df as t1
        left join df_noturno as t2
             on t1.MATRÍCULA = t2.MATRÍCULA 
            and t1.DATA >= t2.DATA 
            and t1.DATA <= t2.[DATA FIM]
       '''
df = pd.read_sql_query(qry, conn)
df.head()

C:\Users\gabri\anaconda3\envs\faculdade\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,Carga,NOTURNO
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97718 entries, 0 to 97717
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MATRÍCULA        97718 non-null  int64  
 1   DATA             97718 non-null  object 
 2   ATRASO           97718 non-null  float64
 3   Jornada - 5x2    2714 non-null   float64
 4   Jornada - 6x1    2714 non-null   float64
 5   Jornada - 12x36  2714 non-null   float64
 6   Jornada - movel  2714 non-null   float64
 7   Jornada - fixo   2714 non-null   float64
 8   Carga            97573 non-null  float64
 9   NOTURNO          48701 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 7.5+ MB


In [14]:
#analisando um caso
df[df['MATRÍCULA'] == 3108]

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,Carga,NOTURNO
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
5,3108,2020-01-23 00:00:00,36.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
6,3108,2020-01-24 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
7,3108,2020-02-03 00:00:00,314.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
8,3108,2020-02-10 00:00:00,150.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
9,3108,2020-02-14 00:00:00,170.0,NaN,NaN,NaN,NaN,NaN,9.0,0.0


In [15]:
df_transporte = pd.read_csv(r'BD Tratado\Transp.csv',sep=';')
df_transporte['DATA FIM'] = pd.to_datetime(df_transporte['DATA FIM'])
df_transporte['DATA'] = pd.to_datetime(df_transporte['DATA'])
df_transporte.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918 entries, 0 to 5917
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   MATRÍCULA         5918 non-null   int64         
 1   NÚM. VIAGENS DIA  5918 non-null   int64         
 2   VALOR POR VIAGEM  5918 non-null   float64       
 3   DATA              5918 non-null   datetime64[ns]
 4   DATA FIM          5918 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2)
memory usage: 231.3 KB


In [16]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_transporte.to_sql('df_transporte', conn, index=False)

qry = '''
    select distinct
        t1.*,
        t2.[NÚM. VIAGENS DIA],
        t2.[VALOR POR VIAGEM]
        
    from
        df as t1
        left join df_transporte as t2
            on t1.MATRÍCULA = t2.MATRÍCULA 
            and t1.DATA >= t2.DATA 
            and t1.DATA <= t2.[DATA FIM]
        '''
df = pd.read_sql_query(qry, conn)
df.head()

C:\Users\gabri\anaconda3\envs\faculdade\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,Carga,NOTURNO,NÚM. VIAGENS DIA,VALOR POR VIAGEM
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97802 entries, 0 to 97801
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MATRÍCULA         97802 non-null  int64  
 1   DATA              97802 non-null  object 
 2   ATRASO            97802 non-null  float64
 3   Jornada - 5x2     2714 non-null   float64
 4   Jornada - 6x1     2714 non-null   float64
 5   Jornada - 12x36   2714 non-null   float64
 6   Jornada - movel   2714 non-null   float64
 7   Jornada - fixo    2714 non-null   float64
 8   Carga             97657 non-null  float64
 9   NOTURNO           48730 non-null  float64
 10  NÚM. VIAGENS DIA  73316 non-null  float64
 11  VALOR POR VIAGEM  73316 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 9.0+ MB


In [18]:
df_func = pd.read_csv(r'BD Tratado\analiseFunc.csv',sep=';')
df_func.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5463 entries, 0 to 5462
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MATRÍCULA      5463 non-null   int64  
 1   Tempo de casa  5463 non-null   float64
 2   SALÁRIO        5463 non-null   float64
 3   SO             3628 non-null   float64
 4   SX             3628 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 213.5 KB


In [19]:
df = df.merge(df_func, on = ['MATRÍCULA'],how='left')
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,Carga,NOTURNO,NÚM. VIAGENS DIA,VALOR POR VIAGEM,Tempo de casa,SALÁRIO,SO,SX
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0


In [20]:
df_demitidos = pd.read_csv(r'BD Tratado\demiticos.csv',sep=';')
df_demitidos['ADMISSÃO'] = pd.to_datetime(df_demitidos['ADMISSÃO'])
df_demitidos['DEMISSÃO'] = pd.to_datetime(df_demitidos['DEMISSÃO'])
df_demitidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3310 entries, 0 to 3309
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   NC Funcionário    3310 non-null   int64         
 1   ADMISSÃO          3310 non-null   datetime64[ns]
 2   CARGO             3310 non-null   object        
 3   DEMISSÃO          3310 non-null   datetime64[ns]
 4   Tipo de Rescisão  3306 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 129.4+ KB


In [21]:
df = df.merge(df_demitidos, how = 'left', left_on='MATRÍCULA', right_on='NC Funcionário')
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,Carga,NOTURNO,...,VALOR POR VIAGEM,Tempo de casa,SALÁRIO,SO,SX,NC Funcionário,ADMISSÃO,CARGO,DEMISSÃO,Tipo de Rescisão
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN


In [22]:
df['Ativo'] = np.where(df['NC Funcionário']>0,0,1)
df = df.drop(['NC Funcionário'],axis=1)
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,Carga,NOTURNO,...,VALOR POR VIAGEM,Tempo de casa,SALÁRIO,SO,SX,ADMISSÃO,CARGO,DEMISSÃO,Tipo de Rescisão,Ativo
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1


In [23]:
df[df['Ativo']==0].head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,Carga,NOTURNO,...,VALOR POR VIAGEM,Tempo de casa,SALÁRIO,SO,SX,ADMISSÃO,CARGO,DEMISSÃO,Tipo de Rescisão,Ativo
69003,231495,2020-01-02 00:00:00,387.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0
69004,231495,2020-01-03 00:00:00,390.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0
69005,231495,2020-01-04 00:00:00,21.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0
69006,231495,2020-01-06 00:00:00,390.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0
69007,231495,2020-01-07 00:00:00,593.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97802 entries, 0 to 97801
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   MATRÍCULA         97802 non-null  int64         
 1   DATA              97802 non-null  object        
 2   ATRASO            97802 non-null  float64       
 3   Jornada - 5x2     2714 non-null   float64       
 4   Jornada - 6x1     2714 non-null   float64       
 5   Jornada - 12x36   2714 non-null   float64       
 6   Jornada - movel   2714 non-null   float64       
 7   Jornada - fixo    2714 non-null   float64       
 8   Carga             97657 non-null  float64       
 9   NOTURNO           48730 non-null  float64       
 10  NÚM. VIAGENS DIA  73316 non-null  float64       
 11  VALOR POR VIAGEM  73316 non-null  float64       
 12  Tempo de casa     97802 non-null  float64       
 13  SALÁRIO           97802 non-null  float64       
 14  SO                8749

In [25]:
df_marcacao = pd.read_excel(r'BD\Qtde de Marcações relógio x manuais - por funcionário e mês.xlsx')
df_marcacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39282 entries, 0 to 39281
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Matrícula       39282 non-null  int64         
 1   MesAno          39282 non-null  datetime64[ns]
 2   QtdeMarcRelog   39282 non-null  int64         
 3   QtdeMarcManual  39282 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.2 MB


In [26]:
df_marcacao['DATA'] = df_marcacao['MesAno'].astype(str)
df_marcacao['DATA FIM'] = np.where(df_marcacao['Matrícula'].shift(-1) == df_marcacao['Matrícula'], df_marcacao['DATA'].shift(-1), '2021-01-31')
df_marcacao['DATA FIM'] = pd.to_datetime(df_marcacao['DATA FIM'])
df_marcacao['DATA'] = pd.to_datetime(df_marcacao['DATA'])
df_marcacao['MANUAL'] = df_marcacao['QtdeMarcManual']/(df_marcacao['QtdeMarcManual']+df_marcacao['QtdeMarcRelog'])
df_marcacao = df_marcacao.drop(['QtdeMarcManual','QtdeMarcRelog', 'MesAno'],axis=1)
df_marcacao = df_marcacao.drop_duplicates()
df_marcacao.head()

,Matrícula,DATA,DATA FIM,MANUAL
0,3108,2020-01-01,2020-02-01,1.0
1,3108,2020-02-01,2020-03-01,1.0
2,3108,2020-03-01,2020-04-01,1.0
3,3108,2020-04-01,2020-05-01,1.0
4,3108,2020-05-01,2020-06-01,1.0


In [27]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_marcacao.to_sql('df_marcacao', conn, index=False)

qry = '''
    select distinct
        t1.*,
        t2.MANUAL
        
    from
        df as t1
        left join df_marcacao as t2
            on t1.MATRÍCULA = t2.Matrícula 
            and t1.DATA >= t2.DATA 
            and t1.DATA <= t2.[DATA FIM]
        '''
df = pd.read_sql_query(qry, conn)
df.info()

C:\Users\gabri\anaconda3\envs\faculdade\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99550 entries, 0 to 99549
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MATRÍCULA         99550 non-null  int64  
 1   DATA              99550 non-null  object 
 2   ATRASO            99550 non-null  float64
 3   Jornada - 5x2     2750 non-null   float64
 4   Jornada - 6x1     2750 non-null   float64
 5   Jornada - 12x36   2750 non-null   float64
 6   Jornada - movel   2750 non-null   float64
 7   Jornada - fixo    2750 non-null   float64
 8   Carga             99405 non-null  float64
 9   NOTURNO           49602 non-null  float64
 10  NÚM. VIAGENS DIA  74650 non-null  float64
 11  VALOR POR VIAGEM  74650 non-null  float64
 12  Tempo de casa     99550 non-null  float64
 13  SALÁRIO           99550 non-null  float64
 14  SO                89101 non-null  float64
 15  SX                89101 non-null  float64
 16  ADMISSÃO          60 non-null     object

In [28]:
df_dist = pd.read_csv(r'BD Tratado\dist.csv',sep=';')
df_dist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MATRÍCULA  3393 non-null   int64  
 1   dist       3393 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 53.1 KB


In [29]:
df = df.merge(df_dist, how='left',on='MATRÍCULA')
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,Carga,NOTURNO,...,SALÁRIO,SO,SX,ADMISSÃO,CARGO,DEMISSÃO,Tipo de Rescisão,Ativo,MANUAL,dist
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,2184.57,3.0,0.0,None,None,None,None,1,1.0,NaN
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,2184.57,3.0,0.0,None,None,None,None,1,1.0,NaN
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,2184.57,3.0,0.0,None,None,None,None,1,1.0,NaN
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,2184.57,3.0,0.0,None,None,None,None,1,1.0,NaN
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,2184.57,3.0,0.0,None,None,None,None,1,1.0,NaN


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99550 entries, 0 to 99549
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MATRÍCULA         99550 non-null  int64  
 1   DATA              99550 non-null  object 
 2   ATRASO            99550 non-null  float64
 3   Jornada - 5x2     2750 non-null   float64
 4   Jornada - 6x1     2750 non-null   float64
 5   Jornada - 12x36   2750 non-null   float64
 6   Jornada - movel   2750 non-null   float64
 7   Jornada - fixo    2750 non-null   float64
 8   Carga             99405 non-null  float64
 9   NOTURNO           49602 non-null  float64
 10  NÚM. VIAGENS DIA  74650 non-null  float64
 11  VALOR POR VIAGEM  74650 non-null  float64
 12  Tempo de casa     99550 non-null  float64
 13  SALÁRIO           99550 non-null  float64
 14  SO                89101 non-null  float64
 15  SX                89101 non-null  float64
 16  ADMISSÃO          60 non-null     object

In [31]:
# base com muitos nulos.
# Retirar as variáveis Jornada e Demissão

df = df.drop(['Jornada - movel','Jornada - fixo','Jornada - 5x2','Jornada - 6x1','Jornada - 12x36','ADMISSÃO','CARGO','DEMISSÃO','Tipo de Rescisão'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99550 entries, 0 to 99549
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MATRÍCULA         99550 non-null  int64  
 1   DATA              99550 non-null  object 
 2   ATRASO            99550 non-null  float64
 3   Carga             99405 non-null  float64
 4   NOTURNO           49602 non-null  float64
 5   NÚM. VIAGENS DIA  74650 non-null  float64
 6   VALOR POR VIAGEM  74650 non-null  float64
 7   Tempo de casa     99550 non-null  float64
 8   SALÁRIO           99550 non-null  float64
 9   SO                89101 non-null  float64
 10  SX                89101 non-null  float64
 11  Ativo             99550 non-null  int64  
 12  MANUAL            99550 non-null  float64
 13  dist              63301 non-null  float64
dtypes: float64(11), int64(2), object(1)
memory usage: 11.4+ MB


In [32]:
df.head(50)

,MATRÍCULA,DATA,ATRASO,Carga,NOTURNO,NÚM. VIAGENS DIA,VALOR POR VIAGEM,Tempo de casa,SALÁRIO,SO,SX,Ativo,MANUAL,dist
0,3108,2020-01-03 00:00:00,1.0,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN
1,3108,2020-01-10 00:00:00,72.0,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN
2,3108,2020-01-20 00:00:00,46.0,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN
3,3108,2020-01-21 00:00:00,47.0,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN
4,3108,2020-01-22 00:00:00,32.0,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN
5,3108,2020-01-23 00:00:00,36.0,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN
6,3108,2020-01-24 00:00:00,46.0,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN
7,3108,2020-02-03 00:00:00,314.0,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN
8,3108,2020-02-10 00:00:00,150.0,9.0,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN
9,3108,2020-02-14 00:00:00,170.0,9.0,0.0,4.0,5.65,195.385271,2184.57,3.0,0.0,1,1.0,NaN


In [33]:
df_horario = pd.read_csv(r'BD Tratado\horario.csv',sep=';')
df_horario['DATA'] = pd.to_datetime(df_horario['DATA'])
df_horario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781042 entries, 0 to 781041
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   MATRÍCULA       781042 non-null  int64         
 1   DATA            781042 non-null  datetime64[ns]
 2   HORÁRIO INÍCIO  781001 non-null  float64       
 3   HORÁRIO FINAL   781001 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 23.8 MB


In [37]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_horario.to_sql('df_horario', conn, index=False)

qry = '''
    select distinct
        t1.*,
        t2.[HORÁRIO INÍCIO],
        t2.[HORÁRIO FINAL]
        
    from
        df as t1
        left join df_horario as t2
            on t1.MATRÍCULA = t2.MATRÍCULA 
            and t1.DATA = t2.DATA 
        '''
df = pd.read_sql_query(qry, conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99550 entries, 0 to 99549
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MATRÍCULA         99550 non-null  int64  
 1   DATA              99550 non-null  object 
 2   ATRASO            99550 non-null  float64
 3   Carga             99405 non-null  float64
 4   NOTURNO           49602 non-null  float64
 5   NÚM. VIAGENS DIA  74650 non-null  float64
 6   VALOR POR VIAGEM  74650 non-null  float64
 7   Tempo de casa     99550 non-null  float64
 8   SALÁRIO           99550 non-null  float64
 9   SO                89101 non-null  float64
 10  SX                89101 non-null  float64
 11  Ativo             99550 non-null  int64  
 12  MANUAL            99550 non-null  float64
 13  dist              63301 non-null  float64
 14  HORÁRIO INÍCIO    99550 non-null  float64
 15  HORÁRIO FINAL     99550 non-null  float64
dtypes: float64(13), int64(2), object(1)
memo

In [38]:
df.to_csv(r'BD Tratado\baseFim.csv',sep=';',index=False)